## Graduate Student Project
#### -Primer Choosing, design and DNA sequence color annotation

### Introduction
In my graduate project, I am trying to create some little tools, which can be used in my real daily working
scenario. As a research assistant in a gene editing lab, I do a lot of molecular bench work related to DNA. One of the most powerful and regular techniques we use daily is called PCR. We use PCR to amplify and confirm DNA sequences. And this graduate project will be closely related to DNA sequences and the technique PCR.

### Backgroud Knowledge of PCR
Polymerase chain reaction (PCR) is a technique used in molecular biology to amplify a single copy or a few copies of a segment of DNA. Generally speaking, DNA has two strands, one is called sense strand and the other is anti-sense strand, and the anti-sense strand has the reverse complementary sequence of sense strand. Each strand has an orientation, and we always write the sense sequence from left to right and name it as 5' to 3'. For anti-sense sequence we also write from left to right but name it as 3' to 5', so it is reversed and complementary.

What is reverse complementary sequence?</p></p>
We have met this several times in previous problem sets. As a quick reminder, there are four kinds of base in DNA strand, represented as 'A', 'T', 'C', 'G', and within the two strands of DNA, 'A' always pairs with 'T', and 'C' pairs with 'G', so 'A' and 'T' are complementary with each other, and the same for 'C' and 'G'. For instance, the reverse complementary sequence of 'ATTGCGGTACG' should be 'CGTACCGCAAT', both of them are in the orientation from 5' to 3'. 

For a successful PCR, we need two primers. The sense primer shares the same sequence with part of the sense strand, and binds to the anti-sense strand, so during the reaction, it can elongate in the orientation of 5' to 3'. And the anti-sense primer shares the same sequence with part of the anti-sense strand, and binds to the sense strand, so during the reaction, it also elongates in the orientation of 5' to 3', but its product is reverse complementary to the product of sense primer. As a result, two DNA product bind to each other as a final product. This process repeats a lot and amplifies the DNA product we want.

### My Aims
* In my lab, every time we design a new primer, we give it a index num and save in our primer list. Up to now we have already generated a list of over 200 primers. So my first aim, is with a new given sequence of DNA, I can search in my primer list, and find sense primers and anti-sense primers that are qualified for a possible PCR.
* Secondly, if there is no qualified primers within our primer list, I'd like to design 5 new primer pairs for this PCR.
* Lastly, for the final DNA sequence of PCR product, I am going to color the four different bases using the same color system as the sequencing results from the sequencing company.

### Python Libraries
I will use 'Biopython' for sequencing analysis, 'Primer3-py' for primer design and 'termcolor' for color annotations.

In [1]:
import os
import csv
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC

### Upload the targeting sequence

Here we have a large fragment called TK vector plasmid, and we are going to use PCR to amplify partial sequence of it. The part we are going to amplify and send sequencing to confirm, is located between the 1520th and 2600th base pair of this plasmid.

In [2]:
with open('TK_vector.txt', mode='r') as f:
    TK_seq = f.read()
    
# full sequence of TK vector plasmid
TK_seq = Seq(TK_seq,IUPAC.unambiguous_dna)
# range of the sequence we'd like to amplify using PCR
target_seq = TK_seq[1520:2600]


### Dictionary of sense primers

Create a dictionary of primers in 5' to 3' orientation, which will be served as sense primers in PCR reaction. The key is the index number, and the value is the sequence of a primer

In [10]:
with open('primer_list.csv', mode='r') as f:
    primer_list = csv.reader(f)
    primer_dict = {}
    for row in primer_list:
        primer_dict[int(row[0])] = Seq(row[1],IUPAC.unambiguous_dna)

#primer_dict

### Dictionary of anti-sense primers

Create a dictionary of anti-sense primers in 5' to 3' orientation, which will be served as anti-sense primers in PCR reaction. The key is the index number, and the value is the sequence of a primer

In [11]:
# Here we only need to get the reverse-complementary sequence of each primer 
# in sense primer dictionary, and keep the same index number
primer_rc_dict = {}
for index in primer_dict.keys():
    my_seq = primer_dict[index]
    primer_rc_dict[int(index)] = my_seq.reverse_complement()
    
#primer_rc_dict

### Create a function that can find all sense primers, which can amplify our target sequence in 5' to 3' orientation

In [5]:
def search_sense(target):
    """
    find out all the primers that can bind to the target seq.
    in sense orientation, and for each qualified primer, it will
    give out the index number of the primer, the seq. of the primer 
    and where the binding starts
    """
    sense_index = []
    for index in primer_dict.keys():
        if primer_dict[index] in target:
            sense_index.append(index)
            
    sense_result = []
    for index in sense_index:
        aseq = primer_dict[index]
        for i in range(0, len(target)-len(aseq)):
            if aseq == target[i:i+len(aseq)]:
                sense_result.append((index, primer_dict[index], i))
    
    return sense_result
search_sense(TK_seq)

[(11, Seq('CACTGCATTCTAGTTGTGGTTTGTCCA', IUPACUnambiguousDNA()), 3418),
 (11, Seq('CACTGCATTCTAGTTGTGGTTTGTCCA', IUPACUnambiguousDNA()), 6459),
 (13, Seq('AACCTCCCCTTCTACGAGCG', IUPACUnambiguousDNA()), 6158),
 (114, Seq('CCTGTACGGCATGGACGAG', IUPACUnambiguousDNA()), 2425),
 (137, Seq('GAGTTCATGCGCTTCAAGGTG', IUPACUnambiguousDNA()), 1055),
 (137, Seq('GAGTTCATGCGCTTCAAGGTG', IUPACUnambiguousDNA()), 1781),
 (138, Seq('GTAGTGTGTGCCCGTCTGTT', IUPACUnambiguousDNA()), 3235),
 (140, Seq('GGCAGCTCCGGCACCGCCTC', IUPACUnambiguousDNA()), 1730),
 (199, Seq('GAAGCTATCTGGTCTCCCTT', IUPACUnambiguousDNA()), 6547),
 (223, Seq('ATCCAGGACACCAGCACGCA', IUPACUnambiguousDNA()), 4479),
 (224, Seq('CTACACTGCCCGGGACAAAT', IUPACUnambiguousDNA()), 5055),
 (231, Seq('TACGAAGTTATATTCAAAAATCTAGACAGC', IUPACUnambiguousDNA()), 3579),
 (233, Seq('CGATATCGCCACCATGGCATCTTATCCAGGAC', IUPACUnambiguousDNA()), 4456),
 (235, Seq('TGAGGCGAACGGCGCTGGAGAGGGCAGAG', IUPACUnambiguousDNA()), 5587)]

### Create a function that can find all anti-sense primers, which can bind to our target sequence in a reverse complemetary format

In [6]:
def search_anti(target):
    """
    find out all the primers that can bind to the target seq.
    in anti-sense orientation, and for each qualified primer, it will
    give out the index number of the primer, the seq. of the primer 
    and where the binding ends
    """
    anti_index = []
    for index in primer_rc_dict.keys():
        if primer_rc_dict[index] in target:
            anti_index.append(index)
            
    anti_result = []
    for index in anti_index:
        aseq = primer_rc_dict[index]
        for i in range(0, len(target)-len(aseq)):
            if aseq == target[i:i+len(aseq)]:
                anti_result.append((index, primer_rc_dict[index], i+len(aseq)-1))
    
    return anti_result
search_anti(TK_seq)

[(67, Seq('AGCAACAGATGGAAGGCCTC', IUPACUnambiguousDNA()), 5995),
 (159, Seq('GATGTCGAAGAGAATCCCGG', IUPACUnambiguousDNA()), 1002),
 (200, Seq('AGGGCGAGGAGGTCATCAAA', IUPACUnambiguousDNA()), 1054),
 (201, Seq('CATGGCACCGGCAGCACCG', IUPACUnambiguousDNA()), 1724),
 (222, Seq('GTGGTGATGACATCTGCCCA', IUPACUnambiguousDNA()), 4842),
 (232, Seq('GCCTACGATATCGCCACCATGGCATCTTATC', IUPACUnambiguousDNA()), 4481),
 (234,
  Seq('CGAGATGGGTGAGGCGAACGGCGCTGGAGAGGGCAG', IUPACUnambiguousDNA()),
  5613),
 (236, Seq('GCTTGGCGTAATCATGGTCA', IUPACUnambiguousDNA()), 7116)]

### Find qualified primer pairs

As I mentioned above, the part we are going to amplify and send sequencing to confirm, is located between the 1520th and 2600th base pair of this plasmid. So we are going to find a sense primer with a start location ahead of 1520 and an anti-sense primer with an end location behind 2600.

In [7]:
print("Primer(s) can be used as sense:")
for item in search_sense(TK_seq):
    if item[2] <= 1520:
        print(item)
        
print("Primer(s) can be used as anti-sense:")
for item in search_anti(TK_seq):
    if item[2] >= 2600:
        print(item)

Primer(s) can be used as sense:
(137, Seq('GAGTTCATGCGCTTCAAGGTG', IUPACUnambiguousDNA()), 1055)
Primer(s) can be used as anti-sense:
(67, Seq('AGCAACAGATGGAAGGCCTC', IUPACUnambiguousDNA()), 5995)
(222, Seq('GTGGTGATGACATCTGCCCA', IUPACUnambiguousDNA()), 4842)
(232, Seq('GCCTACGATATCGCCACCATGGCATCTTATC', IUPACUnambiguousDNA()), 4481)
(234, Seq('CGAGATGGGTGAGGCGAACGGCGCTGGAGAGGGCAG', IUPACUnambiguousDNA()), 5613)
(236, Seq('GCTTGGCGTAATCATGGTCA', IUPACUnambiguousDNA()), 7116)


### Design new primer pairs

Use a library called Primer3-py to design primers.

In [8]:
with open('TK_vector.txt', mode='r') as f:
    TK_seq_new = f.read()
    
import primer3
input_seq = TK_seq_new[1420:2700]

primer = (primer3.bindings.designPrimers(
            {
                'SEQUENCE_ID': 'TK',
                'SEQUENCE_TEMPLATE': input_seq,
                'SEQUENCE_EXCLUDED_REGION': [0, 0] 
            },
            {
                'PRIMER_TASK': 'generic',
                'PRIMER_PICK_LEFT_PRIMER': 1,
                'PRIMER_PICK_INTERNAL_OLIGO': 0,
                'PRIMER_PICK_RIGHT_PRIMER': 1,
                'PRIMER_NUM_RETURN': 5,
                'PRIMER_OPT_SIZE': 20,
                'PRIMER_MIN_SIZE': 18,
                'PRIMER_MAX_SIZE': 25,
                'PRIMER_OPT_TM': 60.0,
                'PRIMER_MIN_TM': 57.0,
                'PRIMER_MAX_TM': 63.0,
                'PRIMER_MIN_GC': 20.0,
                'PRIMER_MAX_GC': 80.0,
                'PRIMER_MAX_POLY_X': 5,
                'PRIMER_SALT_MONOVALENT': 50.0,
                'PRIMER_DNA_CONC': 50.0,
                'PRIMER_MAX_NS_ACCEPTED': 0,
                'PRIMER_MAX_SELF_ANY': 12,
                'PRIMER_MAX_SELF_END': 8,
                'PRIMER_PAIR_MAX_COMPL_ANY': 12,
                'PRIMER_PAIR_MAX_COMPL_END': 8,
                'PRIMER_PRODUCT_SIZE_RANGE': [[len(input_seq)-100,len(input_seq)]],}))

#print(primer)

for i in range(0,5):
    print (i+1,"-", primer['PRIMER_LEFT_{}'.format(i)], primer['PRIMER_LEFT_{}_SEQUENCE'.format(i)],
           primer['PRIMER_RIGHT_{}'.format(i)], primer['PRIMER_RIGHT_{}_SEQUENCE'.format(i)],
          primer['PRIMER_PAIR_{}_PRODUCT_SIZE'.format(i)])


1 - (15, 20) GCAGAAGAAGACCATGGGCT (1246, 20) CGGGCCACAACTCCTCATAA 1232
2 - (14, 20) TGCAGAAGAAGACCATGGGC (1246, 20) CGGGCCACAACTCCTCATAA 1233
3 - (4, 20) GGCCCCGTAATGCAGAAGAA (1246, 20) CGGGCCACAACTCCTCATAA 1243
4 - (4, 20) GGCCCCGTAATGCAGAAGAA (1189, 20) TGAAAGCCATACGGGAAGCA 1186
5 - (15, 20) GCAGAAGAAGACCATGGGCT (1247, 20) ACGGGCCACAACTCCTCATA 1233


### Color DNA sequences

Create a function to color a given DNA sequence using a library called termcolor.

In [9]:
from termcolor import colored

def color_DNA(seq):
    color_dict = {'A':'red','T':'green','C':'blue','G':'yellow'}
    c_str = ''
    for i in seq:
        c_str += colored(i, color_dict[i])
    return c_str
    
# For instance, I am going to color the PCR product of the first primer pair given above
PCR_product = input_seq[15:1246+1]
print(color_DNA(PCR_product))

GCAGAAGAAGACCATGGGCTGGGAGGCCTCCACCGAGCGCCTGTACCCCCGCGACGGCGTGCTGAAGGGCGAGATCCACCAGGCCCTGAAGCTGAAGGACGGCGGCCACTACCTGGTGGAGTTCAAGACCATCTACATGGCCAAGAAGCCCGTGCAACTGCCCGGCTACTACTACGTGGACACCAAGCTGGACATCACCTCCCACAACGAGGACTACACCATCGTGGAACAGTACGAGCGCTCCGAGGGCCGCCACCACCTGTTCCTGGGGCATGGCACCGGCAGCACCGGCAGCGGCAGCTCCGGCACCGCCTCCTCCGAGGACAACAACATGGCCGTCATCAAAGAGTTCATGCGCTTCAAGGTGCGCATGGAGGGCTCCATGAACGGCCACGAGTTCGAGATCGAGGGCGAGGGCGAGGGCCGCCCCTACGAGGGCACCCAGACCGCCAAGCTGAAGGTGACCAAGGGCGGCCCCCTGCCCTTCGCCTGGGACATCCTGTCCCCCCAGTTCATGTACGGCTCCAAGGCGTACGTGAAGCACCCCGCCGACATCCCCGATTACAAGAAGCTGTCCTTCCCCGAGGGCTTCAAGTGGGAGCGCGTGATGAACTTCGAGGACGGCGGTCTGGTGACCGTGACCCAGGACTCCTCCCTGCAGGACGGCACGCTGATCTACAAGGTGAAGATGCGCGGCACCAACTTCCCCCCCGACGGCCCCGTAATGCAGAAGAAGACCATGGGCTGGGAGGCCTCCACCGAGCGCCTGTACCCCCGCGACGGCGTGCTGAAGGGCGAGATCCACCAGGCCCTGAAGCTGAAGGACGGCGGCCACTACCTGGTGGAGTTCAAGACCATCTACATGGCCAAGAAGCCCGTGCAACTGCCCGGCTACTACTACGTGGACACCAAGCTGGACATCACCTCCCACAACGAGGACTACACCATCGTGGAACAGTACGAGCGCTCCGAGGGCCGCCACCACCTGTTCCTGTACGGC

### References

1. http://biopython.org/wiki/Documentation
2. https://libnano.github.io/primer3-py/index.html
3. http://stackoverflow.com/questions/35362345/python-primer3-pcr-primer-design
4. https://pypi.python.org/pypi/termcolor
5. https://en.wikipedia.org/wiki/Polymerase_chain_reaction


### Acknowledgments

I’d like to thank the whole teaching staff of CSCI E-7 Spring 2017, 
who give me such a great class to enter the world of coding. I’m
surprised by myself, when I see how much I can do at the end of 
this semester. 

I’d also like to thank all other students in this class, who helped with
each other a lot through the platform of piazza. I have learnt so 
much from the information and ideas everyone shared or discussed.